In [13]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Fix the seed
tf.random.set_seed(42)
np.random.seed(42)

# === Data Preparation ===
# Generate synthetic multivariate time series data
def generate_time_series_data(n_samples=1000, n_timesteps=50, n_features=5):
    X = np.random.rand(n_samples, n_timesteps, n_features)
    y = np.sum(X[:, -1, :], axis=1, keepdims=True)  # target is sum of last timestep across features
    return X, y

X, y = generate_time_series_data()

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === RNN Model (using 1 feature only) ===
X_rnn_train = X_train[:, :, 0:1]  # Use only 1 feature
X_rnn_test = X_test[:, :, 0:1]

rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(32, return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(1)
])

rnn_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
rnn_model.fit(X_rnn_train, y_train, epochs=10, batch_size=32, verbose=0)
rnn_eval = rnn_model.evaluate(X_rnn_test, y_test, verbose=0)

# === LSTM Model (using all 5 features) ===
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, input_shape=[None, 5]),  # No return_sequences
    tf.keras.layers.Dense(14)
])

lstm_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
lstm_model.fit(X_train, np.repeat(y_train, 14, axis=1), epochs=10, batch_size=32, verbose=0)
lstm_eval = lstm_model.evaluate(X_test, np.repeat(y_test, 14, axis=1), verbose=0)

# === Results ===
print(f"RNN Evaluation - Loss: {rnn_eval[0]:.4f}, MAE: {rnn_eval[1]:.4f}")
print(f"LSTM Evaluation - Loss: {lstm_eval[0]:.4f}, MAE: {lstm_eval[1]:.4f}")


RNN Evaluation - Loss: 0.2896, MAE: 0.4303
LSTM Evaluation - Loss: 0.3248, MAE: 0.4525
